In [15]:

import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.stats import ks_2samp

In [7]:
conn = sqlite3.connect('olist.db')
cursor = conn.cursor()
start_date = datetime(2016, 9, 4)
end_date = datetime(2018, 10, 17)
data_list = []

# Fonction pour générer les périodes d'analyse
def generate_periods(start_date, end_date, increment_days=15):
    periods = []
    current_end_date = start_date + timedelta(days=365)  # Initialiser avec un an après la date de début
    while current_end_date <= end_date:
        periods.append((start_date.strftime('%Y-%m-%d'), current_end_date.strftime('%Y-%m-%d')))
        # Ajouter 15 jours pour le prochain intervalle
        current_end_date += timedelta(days=15)
    return periods

# Générer les périodes
periods = generate_periods(start_date, end_date)

# Exécuter les requêtes pour chaque période
for period_start, period_end in periods:
    query = f"""
    WITH 
        review_scores AS (
        SELECT
            c.customer_unique_id,
            AVG(r.review_score) AS average_review_score
        FROM
            orders o
            JOIN order_reviews r ON o.order_id = r.order_id 
            JOIN customers c ON o.customer_id = c.customer_id
        WHERE
            o.order_purchase_timestamp >= '{period_start}'
            AND o.order_purchase_timestamp <= '{period_end}'
        GROUP BY
            c.customer_unique_id
        ),
        seller_count AS (
        SELECT
            c.customer_unique_id,
            COUNT(DISTINCT i.seller_id) AS nb_of_different_sellers
        FROM
            orders o
            JOIN order_items i ON o.order_id = i.order_id
            JOIN customers c ON o.customer_id = c.customer_id
        WHERE
            o.order_purchase_timestamp >= '{period_start}'
            AND o.order_purchase_timestamp <= '{period_end}'
        GROUP BY
            c.customer_unique_id
        ),
        rfm AS (
        SELECT
            c.customer_unique_id,
            MAX(o.order_purchase_timestamp) AS last_order_date,
            COUNT(o.order_id) AS frequency,
            SUM(i.price) AS monetary
        FROM
            orders o
            JOIN order_items i ON o.order_id = i.order_id
            JOIN customers c ON o.customer_id = c.customer_id
        WHERE
            o.order_purchase_timestamp >= '{period_start}'
            AND o.order_purchase_timestamp <= '{period_end}'
        GROUP BY
            c.customer_unique_id
        ),
        recency_calc AS (
        SELECT
            customer_unique_id,
            JULIANDAY('{period_end}') - JULIANDAY(last_order_date) AS recency
        FROM
            rfm
        ),
        payment_methods AS (
        SELECT
            c.customer_unique_id,
            SUM(CASE WHEN p.payment_type = 'credit_card' THEN p.payment_value ELSE 0 END) / SUM(p.payment_value) AS credit_card_rate,
            SUM(CASE WHEN p.payment_type = 'boleto' THEN p.payment_value ELSE 0 END) / SUM(p.payment_value) AS boleto_rate,
            SUM(CASE WHEN p.payment_type = 'voucher' THEN p.payment_value ELSE 0 END) / SUM(p.payment_value) AS voucher_rate,
            SUM(CASE WHEN p.payment_type = 'debit_card' THEN p.payment_value ELSE 0 END) / SUM(p.payment_value) AS debit_card_rate
        FROM
            orders o
            JOIN order_pymts p ON o.order_id = p.order_id
            JOIN customers c ON o.customer_id = c.customer_id
        WHERE
            p.payment_type != 'not_defined'
            AND o.order_purchase_timestamp >= '{period_start}'
            AND o.order_purchase_timestamp <= '{period_end}'
        GROUP BY
            c.customer_unique_id
        )
    SELECT
        c.customer_unique_id,
        COALESCE(rs.average_review_score, 0) AS average_review_score,
        COALESCE(sc.nb_of_different_sellers, 0) AS nb_of_different_sellers,
        COALESCE(rc.recency, 0) AS recency,
        COALESCE(rfm.frequency, 0) AS frequency,
        COALESCE(rfm.monetary, 0) AS monetary,
        COALESCE(pm.credit_card_rate, 0) AS credit_card_rate,
        COALESCE(pm.boleto_rate, 0) AS boleto_rate,
        COALESCE(pm.voucher_rate, 0) AS voucher_rate,
        COALESCE(pm.debit_card_rate, 0) AS debit_card_rate
    FROM
        customers c
        LEFT JOIN review_scores rs ON c.customer_unique_id = rs.customer_unique_id
        LEFT JOIN seller_count sc ON c.customer_unique_id = sc.customer_unique_id
        LEFT JOIN recency_calc rc ON c.customer_unique_id = rc.customer_unique_id
        LEFT JOIN rfm ON c.customer_unique_id = rfm.customer_unique_id
        LEFT JOIN payment_methods pm ON c.customer_unique_id = pm.customer_unique_id;
    """
    # Exécuter la requête
    # cursor.execute(query)
    data_list.append(pd.read_sql_query(query, conn))

conn.close()

In [8]:
data_list[0].shape

(99441, 10)

In [12]:
grouped_dfs = [
    df.groupby('customer_unique_id').agg({
        'average_review_score': 'mean',
        'nb_of_different_sellers': 'mean',
        'recency': 'mean',
        'frequency': 'mean',
        'monetary': 'mean',
        'credit_card_rate': 'mean',
        'boleto_rate': 'mean',
        'voucher_rate': 'mean',
        'debit_card_rate': 'mean'
    }).loc[lambda x: x['monetary'] != 0]
    for df in data_list
]
print(grouped_dfs[0].shape)
print(grouped_dfs[-1].shape)


(22741, 9)
(95420, 9)


In [17]:
def eval_periods(df_list, features, n_clusters):
    scaler = StandardScaler()
    ari_scores = []
    # ks_p_values = {feature: [] for feature in features}
    for i, df in enumerate(df_list):
        if i == 0:
            X_scaled = scaler.fit_transform(df[features])
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            initial_clusters = kmeans.fit_predict(X_scaled)
            initial_labels = kmeans.labels_
            # initial_features = df.groupby(initial_clusters).mean()
            continue
        X_current = scaler.transform(df[features])
        kmeans_current = kmeans.fit(X_current)
        current_labels = kmeans_current.labels_
        ari = adjusted_rand_score(initial_labels, current_labels)
        ari_scores.append((i, ari))
        print(f"ARI pour la période {i} : {ari}")
        # current_features = df.groupby(kmeans_current.labels_).mean()
        # for feature in features:
            # _, p_value = ks_2samp(initial_features[feature], current_features[feature])
            # ks_p_values[feature].append(p_value)
            # print(f"P-value pour {feature} pour la période {i} : {p_value}")
    print("\nARI Scores pour chaque période par rapport à la période initiale:")
    for period_index, ari in ari_scores:
        print(f"Période {period_index}: ARI = {ari}")
    print("\nP-values pour les caractéristiques:")
    # for feature, p_values in ks_p_values.items():
        # print(f"{feature}: {p_values}")
    
            
        
    
    

In [18]:
eval_periods(grouped_dfs, ['average_review_score', 'nb_of_different_sellers', 'recency', 'frequency', 'monetary', 'credit_card_rate', 'boleto_rate', 'voucher_rate', 'debit_card_rate'], 7)

ValueError: Found input variables with inconsistent numbers of samples: [22741, 24895]